In [86]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from scipy import sparse
import xgboost, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.pipeline import Pipeline, make_pipeline

from importlib import reload 
from sklearn.pipeline import FeatureUnion, make_union

In [82]:
from featurizer import CharGrpExtractorFeaturer, WordExtractorFeaturer
import featurizer

In [81]:
reload(featurizer)

<module 'featurizer' from '/Users/sankoudai/projects/toxic_classify/featurizer.py'>

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [65]:
CountVectorizer?

### example with toxic

In [21]:
type(train[comment_col])

pandas.core.series.Series

In [20]:
for t in train[comment_col]:
    print(t)
    break

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [3]:
label_col = 'toxic'
comment_col = 'comment_text'
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train[comment_col], train[label_col])
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [4]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train[comment_col])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [5]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, ngram_range=(1,1))
tfidf_vect.fit(train[comment_col])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000)
tfidf_vect_ngram.fit(train[comment_col])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# word level tf-idf
tfidf_vect2 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=6000, ngram_range=(1,1))
tfidf_vect2.fit(train[comment_col])
xtrain_tfidf2 =  tfidf_vect2.transform(train_x)
xvalid_tfidf2 =  tfidf_vect2.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram2 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',  ngram_range=(1,2), max_features=6000)
tfidf_vect_ngram2.fit(train[comment_col])
xtrain_tfidf_ngram2 =  tfidf_vect_ngram2.transform(train_x)
xvalid_tfidf_ngram2 =  tfidf_vect_ngram2.transform(valid_x)

/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [17]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', stop_words='english', max_df=0.9, ngram_range=(3,5), max_features=5000)
tfidf_vect_ngram_chars.fit(train[comment_col])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [57]:
badwords = dict((w,1) for w in pd.read_csv('data/badwords.txt', sep='\t')['word'].values)
bwFilter = WordExtractorFeaturer(badwords)
counter = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')

bwcFeaturizer = make_pipeline(bwFilter, counter)
bwcFeaturizer.fit(train[comment_col])
xtrain_bw_count = bwcFeaturizer.transform(train_x)
xvalid_bw_count = bwcFeaturizer.transform(valid_x)

In [83]:
badchars = dict((w,1) for w in pd.read_csv('data/bad_chargrps.txt', sep='\t')['chargrp'].values)
badcharFilter = CharGrpExtractorFeaturer(badchars)
charCounter = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
bchf = make_pipeline(badcharFilter, charCounter)
bchf.fit(train[comment_col])
xtrain_bc_count = bchf.transform(train_x)
xvalid_bc_count = bchf.transform(valid_x)

In [87]:
uf = make_union(bchf, tfidf_vect, make_pipeline(tfidf_vect, nbfeaturer))
xtrain_u = uf.transform(train_x)
xvalid_u = uf.transform(valid_x)

/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [89]:
xtrain_u.shape

(119678, 15817)

In [6]:
from sklearn.base import BaseEstimator

class NBFeaturer(BaseEstimator):
    def __init__(self, alpha):
        self.alpha = alpha

    def preprocess_x(self, x, r):
        return x.multiply(r)

    def pr(self, x, y_i, y):
        p = x[y==y_i].sum(0)
        return (p+self.alpha) / (p.sum()+self.alpha)

    def fit(self, x, y=None):
        self._r = sparse.csr_matrix(np.log(self.pr(x,1,y) / self.pr(x,0,y)))
        return self

    def transform(self, x):
        x_nb = self.preprocess_x(x, self._r)
        return x_nb

nbfeaturer = NBFeaturer(0.2)
nbfeaturer.fit(xtrain_tfidf, train_y)
xtrain_nb = nbfeaturer.transform(xtrain_tfidf)
xvalid_nb = nbfeaturer.transform(xvalid_tfidf)


nbfeaturer2 = NBFeaturer(0.2)
nbfeaturer2.fit(xtrain_tfidf2, train_y)
xtrain_nb2 = nbfeaturer2.transform(xtrain_tfidf2)
xvalid_nb2 = nbfeaturer2.transform(xvalid_tfidf2)

(119678, 6000)

In [7]:
from gensim.models import KeyedVectors
# load the google word2vec model
filename = 'data/GoogleNews-vectors-negative300.bin'
goog_em_model = KeyedVectors.load_word2vec_format(filename, binary=True)

tfidf_vect_inf = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=20000, ngram_range=(1,1))
tfidf_vect_inf.fit(train[comment_col])
def trans2wv_avg(wc_xs, em_model, count_vect):
    wv_size = em_model.wv.vector_size
    doc_cnt = wc_xs.get_shape()[0]
    xs = np.zeros((doc_cnt, wv_size), dtype=np.float64)
    words = count_vect.get_feature_names()
    for doc_ind in range(doc_cnt):
        doc_vec = wc_xs.getrow(doc_ind)
        doc_em = np.zeros(wv_size)
        doc_wc = 0
        for word_ind, wc in zip(doc_vec.indices, doc_vec.data):
            if word_ind >= len(words):continue
            word = words[word_ind]
            if word in em_model:
                doc_em += em_model[word]*wc
                doc_wc += wc
        doc_em = doc_em / doc_wc
        xs[doc_ind] = doc_em
    return xs
                
xtrain_g300_avg = trans2wv_avg(xtrain_count, goog_em_model, count_vect)
xvalid_g300_avg = trans2wv_avg(xvalid_count, goog_em_model, count_vect)
xtrain_g300_avg_tfidf = trans2wv_avg(xtrain_count, goog_em_model, tfidf_vect_inf)
xvalid_g300_avg_tfidf = trans2wv_avg(xvalid_count, goog_em_model, tfidf_vect_inf)

def trans2wv_max(wc_xs, em_model):
    wv_size = em_model.wv.vector_size
    doc_cnt = wc_xs.get_shape()[0]
    xs = np.zeros((doc_cnt, wv_size), dtype=np.float64)
    words = count_vect.get_feature_names()
    for doc_ind in range(doc_cnt):
        doc_vec = wc_xs.getrow(doc_ind)
        doc_em = np.zeros(wv_size)
        for word_ind, wc in zip(doc_vec.indices, doc_vec.data):
            if word_ind >= len(words):continue
            word = words[word_ind]
            if word in em_model:
                doc_em = np.max(np.array([em_model[word], doc_em]), axis=0)
        doc_em = doc_em
        xs[doc_ind] = doc_em
    return xs

def trans2wv_max2(wc_xs, em_model, count_vect):
    wv_size = em_model.wv.vector_size
    doc_cnt = wc_xs.get_shape()[0]
    xs = np.zeros((doc_cnt, wv_size), dtype=np.float64)
    words = count_vect.get_feature_names()
    for doc_ind in range(doc_cnt):
        doc_vec = wc_xs.getrow(doc_ind)
        doc_em = np.zeros(wv_size)
        for word_ind, wc in zip(doc_vec.indices, doc_vec.data):
            if word_ind >= len(words):continue
            word = words[word_ind]
            if word in em_model:
                doc_em = np.max(np.array([em_model[word]*wc, doc_em]), axis=0)
        doc_em = doc_em
        xs[doc_ind] = doc_em
    return xs

xtrain_g300_max_tfidf = trans2wv_max2(xtrain_count, goog_em_model, tfidf_vect_inf)
xvalid_g300_max_tfidf = trans2wv_max2(xvalid_count, goog_em_model, tfidf_vect_inf)
xtrain_g300_max = trans2wv_max(xtrain_count, goog_em_model)
xvalid_g300_max = trans2wv_max(xvalid_count, goog_em_model)


/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [60]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_label=valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_label)

In [58]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf2, train_y, xvalid_tfidf2)
print("NB, WordLevel TF-IDF 6000: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram2, train_y, xvalid_tfidf_ngram2)
print("NB, N-Gram Vectors 6000: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, N-Gram char: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_bw_count, train_y, xvalid_bw_count)
print("NB, bad word count: ", accuracy)

NB, Count Vectors:  0.940190008272128
NB, WordLevel TF-IDF:  0.9488632090843006
NB, WordLevel TF-IDF 6000:  0.9491890807911162
NB, N-Gram Vectors:  0.9475597222570376
NB, N-Gram Vectors 6000:  0.9484120021056326
NB, N-Gram char:  0.9372320958564159
NB, bad word count:  0.9132178577695335


In [84]:
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_bc_count, train_y, xvalid_bc_count)
print("NB, bad char count: ", accuracy)

NB, bad word count:  0.9268042012383125


# Logistic Regression

In [146]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf2, train_y, xvalid_tfidf2)
print("LR, WordLevel TF-IDF 6000: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram2, train_y, xvalid_tfidf_ngram2)
print("LR, N-Gram Vectors 6000: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_nb, train_y, xvalid_nb)
print("LR, nb Vectors: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_nb2, train_y, xvalid_nb2)
print("LR, nb Vectors 6000: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print("LR, nb chars 6000: ", accuracy)

LR, Count Vectors:  0.9505427017271201
LR, WordLevel TF-IDF:  0.9585390920712907
LR, WordLevel TF-IDF 6000:  0.9586894943975134
LR, N-Gram Vectors:  0.9560323866342466
LR, N-Gram Vectors 6000:  0.9559571854711353
LR, nb Vectors:  0.960544456420926
LR, nb Vectors 6000:  0.9607951269646304


In [61]:
accuracy = train_model(linear_model.LogisticRegression(),xtrain_bw_count, train_y, xtrain_bw_count, train_y)
print("LR, badword count  train: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(),xtrain_bw_count, train_y, xvalid_bw_count)
print("LR, badword count  valid: ", accuracy)

LR, badword count  train:  0.9227928274202444
LR, badword count  valid:  0.9135437294763492


In [85]:
accuracy = train_model(linear_model.LogisticRegression(),xtrain_bc_count, train_y, xvalid_bc_count)
print("LR, badword count  valid: ", accuracy)

LR, badword count  valid:  0.9396385330759782


In [88]:
accuracy = train_model(linear_model.LogisticRegression(),xtrain_u, train_y, xvalid_u)
print("LR, ultra  valid: ", accuracy)

LR, ultra  valid:  0.9556062467099491


In [62]:
type(xtrain_bw_count)

scipy.sparse.csr.csr_matrix

In [33]:
xtrain_g300_avg[np.isnan(xtrain_g300_avg)] = 0 
xvalid_g300_avg[np.isnan(xvalid_g300_avg)] = 0
xtrain_g300_avg_tfidf[np.isnan(xtrain_g300_avg_tfidf)] = 0 
xvalid_g300_avg_tfidf[np.isnan(xvalid_g300_avg_tfidf)] = 0
xtrain_g300_max[np.isnan(xtrain_g300_max)] = 0 
xvalid_g300_max[np.isnan(xvalid_g300_max)] = 0
xtrain_g300_max_tfidf[np.isnan(xtrain_g300_max_tfidf)] = 0 
xvalid_g300_max_tfidf[np.isnan(xvalid_g300_max_tfidf)] = 0
accuracy = train_model(linear_model.LogisticRegression(), xtrain_g300_avg, train_y, xvalid_g300_avg)
print("LR, g300_avg: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_g300_avg_tfidf, train_y, xvalid_g300_avg_tfidf)
print("LR, g300_avg_tfidf: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_g300_max, train_y, xvalid_g300_max)
print("LR, g300_max: ", accuracy)

accuracy = train_model(linear_model.LogisticRegression(), xtrain_g300_max_tfidf, train_y, xvalid_g300_max_tfidf)
print("LR, g300_max_tfidf: ", accuracy)

LR, g300_avg:  0.9440252675908054
LR, g300_avg_tfidf:  0.9028150302058006
LR, g300_max:  0.9395633319128669
LR, g300_max_tfidf:  0.9033414383475797


In [156]:
lr = linear_model.LogisticRegression()
lr.fit(xtrain_g300_avg, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [173]:
xtrain_g300_avg[0].shape

(300,)

In [170]:
xtrain_g300_avg[0] == xtrain_g300_avg[100000]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [153]:
0.9062241496001805+0.09377585039981952

1.0

# SVM

In [12]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_g300_avg, train_y, xvalid_g300_avg)
print("SVM, word Vectors: ", accuracy)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# xgb

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

In [11]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_g300_avg, train_y, xvalid_g300_avg)
print("g300_avg: ", accuracy)

g300_avg:  0.9449276815481413


/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### EXP

In [93]:
print(cv.data)
print(cv.indices)

[1 1 1 1 2 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1]
[  1347   1931   8250   9236   9776  10494  14583  16167  36412  48592
  50703  54515  57036  72714  81150  82385  94352  99480 101111 101375
 101521 111959 116752 117322 117382 119640 120071 120099 128334 134929
 148651 151286 154445 155917 157498 157823 162888 165753 166256 167792
 168831 174476 179046 179475 179603 180801 181898 182849 185881 185954]


In [95]:
len(xtrain_count.get_shape()[0])

TypeError: object of type 'int' has no len()

In [88]:
len(count_vect.get_feature_names())

190339

In [ ]:
xtrain_count

In [81]:
i = xtrain_tfidf[0:2]
print(i.data.shape)
print(i.indices)
print(i.shape)
print(i.getnnz())
print(i.getrow(0).indices)

(67,)
[4987 4983 4916 4896 4870 4834 4825 4809 4685 4529 4472 4405 4266 4249
 4197 4143 4070 3998 3609 3157 3156 3080 3076 2986 2779 2762 2709 2608
 2320 2301 2093 1628 1568 1402  476  428  286  278  275  250   38 4948
 4863 4826 4529 4483 4480 4470 4431 4278 3243 3101 2770 2642 2522 2424
 2320 2099 1968 1256 1213 1180  702  694  476  455  444]
(2, 5000)
67
[4987 4983 4916 4896 4870 4834 4825 4809 4685 4529 4472 4405 4266 4249
 4197 4143 4070 3998 3609 3157 3156 3080 3076 2986 2779 2762 2709 2608
 2320 2301 2093 1628 1568 1402  476  428  286  278  275  250   38]


In [65]:
for i in xtrain_tfidf[0:1].:
    print(i)

  (0, 4987)	0.0746800166946854
  (0, 4983)	0.05142511992529977
  (0, 4916)	0.07179042201412006
  (0, 4896)	0.08189734307897052
  (0, 4870)	0.2045108638305972
  (0, 4834)	0.1958609889056362
  (0, 4825)	0.19226344567547962
  (0, 4809)	0.22614994704079744
  (0, 4685)	0.1009733781344479
  (0, 4529)	0.08974541162023222
  (0, 4472)	0.10707673173031468
  (0, 4405)	0.12067110382430597
  (0, 4266)	0.15904668705810476
  (0, 4249)	0.1963898778606448
  (0, 4197)	0.173885191407347
  (0, 4143)	0.1880355663776124
  (0, 4070)	0.16719762487808212
  (0, 3998)	0.09442547743100704
  (0, 3609)	0.17680994075753453
  (0, 3157)	0.26611943189090215
  (0, 3156)	0.05112122162733816
  (0, 3080)	0.0882276392834165
  (0, 3076)	0.18951909612520346
  (0, 2986)	0.23617933604339875
  (0, 2779)	0.16301250581842372
  (0, 2762)	0.13558338613184528
  (0, 2709)	0.14400162515240644
  (0, 2608)	0.2024082270061789
  (0, 2320)	0.047939090005863834
  (0, 2301)	0.09965399084632623
  (0, 2093)	0.11767288799617458
  (0, 1628)	0.119

In [55]:
len(model.word_vec('curse'))

300

In [19]:
token = text.Tokenizer()
token.fit_on_texts(train[comment_col])

In [26]:
sequence.pad_sequences(token.texts_to_sequences(train_x[0:2]), maxlen=100)

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,    121,    154,      5,
           250,   3637,    212,      5,    702,  25333,     64,    253,
             7,    477,     70,      6,   6794,     21,     46,   1179,
           300,      2,    885,      3,   2243,    593,     76,     42,
          1376,  17730,   1587,     28,    126,      4,   1035,    334,
            20,   1494,   1449,   8799,      4,   4376,  29790,      2,
          7331,     88,    212,      4,   1955,  25685,   5512,   1668,
          2155,   5169,   1754,   3881],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,

136467    Go take a long walk off a short pier. \n\nSee ...
Name: comment_text, dtype: object

In [23]:
token.index_word[121]

'go'

In [47]:
tfidf_vect.get_feature_names().index('damn')

1347

In [185]:
def trans2wv_avg(wc_xs, em_model, count_vect):
    wv_size = em_model.wv.vector_size
    doc_cnt = wc_xs.get_shape()[0]
    xs = np.zeros((doc_cnt, wv_size), dtype=np.float64)
    words = count_vect.get_feature_names()
    for doc_ind in range(doc_cnt):
        doc_vec = wc_xs.getrow(doc_ind)
        doc_em = np.zeros(wv_size)
        doc_wc = 0
        for word_ind, wc in zip(doc_vec.indices, doc_vec.data):
            if word_ind >= len(words):continue
            word = words[word_ind]
            if word in em_model:
                doc_em += em_model[word]*wc
                doc_wc += wc
        doc_em = doc_em / doc_wc
        xs[doc_ind] = doc_em
    return xs
                
xtrain_g300_avg = trans2wv_avg(xtrain_count, goog_em_model, count_vect)

/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [177]:
(xtrain_count[0,:]==xtrain_count[10,:]).sum()

/Users/sankoudai/Library/Python/3.6/lib/python/site-packages/scipy/sparse/compressed.py:226: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


190285

In [186]:
t = xtrain_g300_avg[0]
for i in range(len(xtrain_g300_avg)):
    t2 = xtrain_g300_avg[i]
    if (t != t2).sum() != 0:
        print(i)
        break

1


In [29]:
class TextFeaturer(BaseEstimator):
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        df = pd.DataFrame({"text":x})
        df['char_count'] = df['text'].apply(len)
        df['word_count'] = df['text'].apply(lambda x: len(x.split()))
        df['word_density'] = df['char_count'] / (df['word_count']+1)
        df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
        df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
        df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
        
        pos_family = {
            'noun' : ['NN','NNS','NNP','NNPS'],
            'pron' : ['PRP','PRP$','WP','WP$'],
            'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
            'adj' :  ['JJ','JJR','JJS'],
            'adv' : ['RB','RBR','RBS','WRB']
        }

        # function to check and get the part of speech tag count of a words in a given sentence
        def check_pos_tag(x, flag):
            cnt = 0
            try:
                wiki = textblob.TextBlob(x)
                for tup in wiki.tags:
                    ppo = list(tup)[1]
                    if ppo in pos_family[flag]:
                        cnt += 1
            except:
                pass
            return cnt

        df['noun_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'noun'))/df['word_count']
        df['verb_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'verb'))/df['word_count']
        df['adj_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'adj'))/df['word_count']
        df['adv_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'adv'))/df['word_count']
        df['pron_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'pron'))/df['word_count']
        df['noun_count'] = df['text'].apply(lambda x: check_pos_tag(x, 'noun'))
#         df = df.drop("text", axis=1)
#         return df.values
        return df

In [30]:
text_featurer = TextFeaturer()
t = text_featurer.transform(train_x)

In [34]:
t = pd.DataFrame({"text":train_x})